In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns






In [14]:
import re

import traceback

import string

import os

from pandas import Series

import scipy.stats.stats as stats

import pandas.core.algorithms as algos


In [16]:
import plotly.plotly as py

import plotly.graph_objs as go

import plotly

import plotly.figure_factory as ff

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

plotly.offline.init_notebook_mode(connected=True)

ModuleNotFoundError: No module named 'plotly'

In [17]:
print(os.listdir("../input"))

import matplotlib.pyplot as plt

%matplotlib inline

from pylab import rcParams

rcParams['figure.figsize'] = 14, 8

import warnings

warnings.filterwarnings('ignore')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input'

In [3]:
max_bin = 20

force_bin = 3

In [4]:
df=pd.read_csv("C:\\Users\\DELL\\Desktop\\CAPSTONE PROJECT 10\\online_shoppers_intention.csv")

In [5]:
data=df.copy()

In [6]:
data1=pd.get_dummies(data=data, columns=['Month', 'VisitorType'])
data1.head(5)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
def mono_bin(Y, X, n = max_bin):
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
        
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

In [18]:
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

In [19]:
def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [20]:
final_iv, IV = data_vars(data1,data1.Revenue)

In [21]:
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,Administrative,0.000000,1.000000,7122,805.0,0.113030,6317.0,0.886970,0.421908,0.606122,-0.362294,0.137421
1,Administrative,2.000000,27.000000,5208,1103.0,0.211790,4105.0,0.788210,0.578092,0.393878,0.383691,0.137421
2,Administrative_Duration,0.000000,7.500000,6167,586.0,0.095022,5581.0,0.904978,0.307128,0.535502,-0.555940,0.218286
3,Administrative_Duration,7.600000,3398.750000,6163,1322.0,0.214506,4841.0,0.785494,0.692872,0.464498,0.399888,0.218286
4,Informational,0.000000,1.000000,10740,1531.0,0.142551,9209.0,0.857449,0.802411,0.883612,-0.096397,0.050803
5,Informational,2.000000,24.000000,1590,377.0,0.237107,1213.0,0.762893,0.197589,0.116388,0.529257,0.050803
6,Informational_Duration,0.000000,1.000000,9928,1343.0,0.135274,8585.0,0.864726,0.703878,0.823738,-0.157247,0.081031
7,Informational_Duration,1.500000,2549.375000,2402,565.0,0.235221,1837.0,0.764779,0.296122,0.176262,0.518800,0.081031
8,ProductRelated,0.000000,3.000000,1583,64.0,0.040430,1519.0,0.959570,0.033543,0.145749,-1.469061,0.383754
9,ProductRelated,4.000000,6.000000,1182,62.0,0.052453,1120.0,0.947547,0.032495,0.107465,-1.196086,0.383754


In [22]:
data = [go.Bar(
            x=final_iv['VAR_NAME'],
            y=final_iv['WOE'],
            text=final_iv['VAR_NAME'],
            marker=dict(
            color='rgb(158,20,25)',
            line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
    )]


layout = go.Layout(
    title='Weight of Evidence(WOE)',
        xaxis=dict(
        title='Features',
            tickangle=-45,
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Weight of Evidence(WOE)',
        titlefont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
)

plotly.offline.iplot({
    "data": data,'layout':layout
})

NameError: name 'go' is not defined

In [23]:
IV.sort_values('IV',ascending=False)

,VAR_NAME,IV
18,PageValues,2.379658
4,ExitRates,0.818270
20,ProductRelated_Duration,0.459723
19,ProductRelated,0.383754
1,Administrative_Duration,0.218286
14,Month_Nov,0.162315
0,Administrative,0.137421
2,BounceRates,0.098070
6,Informational_Duration,0.081031
26,VisitorType_Returning_Visitor,0.070821


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [25]:
X = data1.drop(['OperatingSystems','TrafficType','Month_Dec','Month_Oct','Weekend','Month_June','Month_Sep','Browser','Month_Aug','Region','VisitorType_Other','Month_Jul','SpecialDay','Revenue'],axis=1)
y = data1['Revenue']

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [27]:
model = LogisticRegression()

In [28]:
model.fit(x_train, y_train)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
y_test_predicted = model.predict(x_test)
y_test_predicted

array([False, False, False, ..., False, False, False])

In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_test_predicted)
confusion_matrix

array([[3008,   69],
       [ 391,  231]], dtype=int64)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score = accuracy_score(y_test, y_test_predicted)
accuracy_score

0.8756420654230873

In [32]:
from sklearn.metrics import classification_report

In [33]:
print(classification_report(y_test, y_test_predicted, target_names=['Yes', 'No']))

              precision    recall  f1-score   support

         Yes       0.88      0.98      0.93      3077
          No       0.77      0.37      0.50       622

    accuracy                           0.88      3699
   macro avg       0.83      0.67      0.72      3699
weighted avg       0.87      0.88      0.86      3699



In [34]:
def Models(x_train,y_train,x_test,y_test):
    
    x_train=x_train
    y_train=y_train
    x_test=x_test
    y_test=y_test

    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.tree import DecisionTreeClassifier
    from sklearn import metrics
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import BernoulliNB, MultinomialNB
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier

    
    print("LOGISTIC REGRESSION")
    lg = LogisticRegression()
    lg.fit(x_train, y_train)
    ypred = lg.predict(x_test)
    confusion_matrix = confusion_matrix(y_test, ypred)
    print("Confusion matrix for Logistic:",confusion_matrix)
    accuracy_score = accuracy_score(y_test, ypred)
    print("Accuracy score for Logistic:",accuracy_score)
    print("Classification Report for Logistic ",classification_report(y_test, ypred, target_names=['Yes', 'No']))

    print("DECISION TREE")
    dt=DecisionTreeClassifier(criterion = 'gini',max_depth=4)
    dt.fit(x_train,y_train)
    ypred2=dt.predict(x_test)
    cm2=metrics.confusion_matrix(y_test,ypred2)
    print("Confusion matrix for Decision Tree:",cm2)
    acc2=metrics.accuracy_score(y_test,ypred2)
    print("Accuracy score for Decision Tree:",acc2)
    cr2=metrics.classification_report(y_test,ypred2)
    print("Classification Report for Decision tree",cr2)
    
    print("Random forest Classifier")
    rfcl = RandomForestClassifier(n_estimators = 6)
    rfcl = rfcl.fit(x_train, y_train)
    ypred6=rfcl.predict(x_test)
    cm6=metrics.confusion_matrix(y_test,ypred6)
    print("Confusion matrix for random forest Tree:",cm6)
    acc6=metrics.accuracy_score(y_test,ypred6)
    print("Accuracy score for random forest Tree:",acc6)
    cr6=metrics.classification_report(y_test,ypred6)
    print("Classification Report for Random forest",cr6)
    
    print("Gradient Boosting")
    gbcl = GradientBoostingClassifier(n_estimators = 51)
    gbcl.fit(x_train,y_train)
    ypred7=gbcl.predict(x_test)
    cm7=metrics.confusion_matrix(y_test,ypred7)
    print("Confusion matrix for Gradient boosting:",cm7)
    acc7=metrics.accuracy_score(y_test,ypred7)
    print("Accuracy score for Gradient Boosting:",acc7)
    cr7=metrics.classification_report(y_test,ypred7)
    print("Classification Report for Gradient Boosting ",cr7)

    print("Naive Bayes Gaussian")
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    ypred3=gnb.predict(x_test)
    cm3=metrics.confusion_matrix(y_test,ypred3)
    print("Confusion matrix for Gaussian:",cm3)
    acc3=metrics.accuracy_score(y_test,ypred3)
    print("Accuracy score for Gaussian:",acc3)
    cr3=metrics.classification_report(y_test,ypred3)
    print("Classification Report for Gaussian",cr3)

    print("Naive Bayes Bernoulli")
    bnb = BernoulliNB()
    bnb.fit(x_train, y_train)
    ypred4=bnb.predict(x_test)
    cm4=metrics.confusion_matrix(y_test,ypred4)
    print("Confusion matrix for Bernoulli",cm4)
    acc4=metrics.accuracy_score(y_test,ypred4)
    print("Accuracy score for Bernoulli:",acc4)
    cr4=metrics.classification_report(y_test,ypred4)
    print("Classification Report for Bernoulli:",cr4)

    print("Naive Bayes Multinomial")
    mnb = MultinomialNB()
    mnb.fit(x_train, y_train)
    ypred5=mnb.predict(x_test)
    cm5=metrics.confusion_matrix(y_test,ypred5)
    print("Confusion matrix for Multinomial:",cm5)
    acc=metrics.accuracy_score(y_test,ypred5)
    print("Accuracy score for Multinomial:",acc)
    cr5=metrics.classification_report(y_test,ypred5)
    print("Classification Report for Multinomial:",cr5)
    
    return

In [35]:
Models(x_train,y_train,x_test,y_test)

LOGISTIC REGRESSION


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Confusion matrix for Logistic: [[3008   69]
 [ 391  231]]
Accuracy score for Logistic: 0.8756420654230873
Classification Report for Logistic                precision    recall  f1-score   support

         Yes       0.88      0.98      0.93      3077
          No       0.77      0.37      0.50       622

    accuracy                           0.88      3699
   macro avg       0.83      0.67      0.72      3699
weighted avg       0.87      0.88      0.86      3699

DECISION TREE
Confusion matrix for Decision Tree: [[2947  130]
 [ 275  347]]
Accuracy score for Decision Tree: 0.8905109489051095
Classification Report for Decision tree               precision    recall  f1-score   support

       False       0.91      0.96      0.94      3077
        True       0.73      0.56      0.63       622

    accuracy                           0.89      3699
   macro avg       0.82      0.76      0.78      3699
weighted avg       0.88      0.89      0.88      3699

Random forest Classifier
Confusion

In [ ]:
from collections import OrderedDict
import pandas as pd
import sys
%matplotlib inline
    
def plot_pandas_style(styler):
    from IPython.core.display import HTML
    html = '\n'.join([line.lstrip() for line in styler.render().split('\n')])
    return HTML(html)

def highlight_max(s,color='yellow'):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: {}'.format(color) if v else '' for v in is_max]

def decile_labels(agg1,label,color='skyblue'):
    agg_dummy = pd.DataFrame(OrderedDict((('TOTAL',0),('TARGET',0),('NONTARGET',0),('PCT_TAR',0),('CUM_TAR',0),('CUM_NONTAR',0),('DIST_TAR',0),('DIST_NONTAR',0),('SPREAD',0))),index=[0])
    agg1 = agg1.append(agg_dummy).sort_index()
    agg1.index.name = label
    agg1 = agg1.style.apply(highlight_max, color = 'yellow', subset=['SPREAD'])
    agg1.bar(subset=['TARGET'], color='{}'.format(color))
    agg1.bar(subset=['TOTAL'], color='{}'.format(color))
    agg1.bar(subset=['PCT_TAR'], color='{}'.format(color))
    return(agg1)

def deciling(data,decile_by,target,nontarget):
    inputs = list(decile_by)
    inputs.extend((target,nontarget))
    decile = data[inputs]
    grouped = decile.groupby(decile_by)
    agg1 = pd.DataFrame({},index=[])
    agg1['TOTAL'] = grouped.sum()[nontarget] + grouped.sum()[target]
    agg1['TARGET'] = grouped.sum()[target]
    agg1['NONTARGET'] = grouped.sum()[nontarget]
    agg1['PCT_TAR'] = grouped.mean()[target]*100
    agg1['CUM_TAR'] = grouped.sum()[target].cumsum()
    agg1['CUM_NONTAR'] = grouped.sum()[nontarget].cumsum()
    agg1['DIST_TAR'] = agg1['CUM_TAR']/agg1['TARGET'].sum()*100
    agg1['DIST_NONTAR'] = agg1['CUM_NONTAR']/agg1['NONTARGET'].sum()*100
    agg1['SPREAD'] = (agg1['DIST_TAR'] - agg1['DIST_NONTAR'])
    agg1 = decile_labels(agg1,'DECILE',color='skyblue')
    return(plot_pandas_style(agg1))

In [ ]:
def scoring(features,clf,target):
    score = pd.DataFrame(clf.predict_proba(features)[:,1], columns = ['SCORE'])
    score['DECILE'] = pd.qcut(score['SCORE'].rank(method = 'first'),10,labels=range(10,0,-1))
    score['DECILE'] = score['DECILE'].astype(float)
    score['TARGET'] = target
    score['NONTARGET'] = 1 - target
    return(score)

In [ ]:
scores_train = scoring(x_train,rfcl,y_train)
scores_test = scoring(x_test,rfcl,y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfcl = RandomForestClassifier(n_estimators = 6)
rfcl = rfcl.fit(x_train, y_train)    
rfcl_pred=rfcl.predict(x_test)

In [ ]:
deciling(data1,['DECILE'],'y','X')

In [ ]:
deciling(data1,['DECILE'],'TARGET','NONTARGET')

In [36]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [37]:
from imblearn.over_sampling import SMOTE

In [38]:
y_train.value_counts()

False    7345
True     1286
Name: Revenue, dtype: int64

In [39]:
smt=SMOTE()
x_train,y_train=smt.fit_sample(x_train,y_train)

In [40]:
np.bincount(y_train)

array([7345, 7345], dtype=int64)

In [41]:
x_test,y_test=smt.fit_sample(x_test,y_test)

In [42]:
np.bincount(y_test)

array([3077, 3077], dtype=int64)

In [43]:
Models(x_train,y_train,x_test,y_test)

LOGISTIC REGRESSION


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Confusion matrix for Logistic: [[2739  338]
 [ 712 2365]]
Accuracy score for Logistic: 0.829379265518362
Classification Report for Logistic                precision    recall  f1-score   support

         Yes       0.79      0.89      0.84      3077
          No       0.87      0.77      0.82      3077

    accuracy                           0.83      6154
   macro avg       0.83      0.83      0.83      6154
weighted avg       0.83      0.83      0.83      6154

DECISION TREE
Confusion matrix for Decision Tree: [[2779  298]
 [ 251 2826]]
Accuracy score for Decision Tree: 0.9107897302567436
Classification Report for Decision tree               precision    recall  f1-score   support

       False       0.92      0.90      0.91      3077
        True       0.90      0.92      0.91      3077

    accuracy                           0.91      6154
   macro avg       0.91      0.91      0.91      6154
weighted avg       0.91      0.91      0.91      6154

Random forest Classifier
Confusion 